In [1]:
import torch
import torch.nn as nn

import math

from transformers import AutoTokenizer, AutoModelForCausalLM

model = 'distilbert/distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model)
transformer = AutoModelForCausalLM.from_pretrained(model)

In [2]:
texts = ['Hello, ML&DS Community!', 'hi bombo anton']

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
tokenized = tokenizer(texts, padding = True, return_tensors='pt')

inp = tokenized['input_ids']
attn = tokenized['attention_mask']

In [3]:
from modules import Decoder
decoder = Decoder(tokenizer)
decoder(inp, attn = attn).shape

torch.Size([2, 7, 64])

In [5]:
decoder.generate('antoha kak dela')

'5<ITB"^8^%'